In [1]:
import pandas as pd
from scipy.stats import zscore
from sklearn.metrics.pairwise import euclidean_distances
from src.models import db, Laptop, Cpu, Gpu, Windows as Wind, Application as App
from src import app

### GET ALL LAPTOP FROM DATABASE

In [2]:
laptop_query = db.select(Laptop.id, Laptop.name, Laptop.hddStorage,
                      Laptop.ssdStorage, Laptop.ram, Cpu.maxSpeed,
                      Cpu.cores, Gpu.maxSpeed.label("gpuMaxSpeed"), Gpu.memory.label("gpuMemory"), Gpu.directX, Gpu.openGl, Wind.buildNumber).join(Laptop.cpu).join(Laptop.gpu).join(Laptop.windows)

with app.app_context():
    laptops_full = pd.read_sql(laptop_query, con=db.engine)
laptops_full["totalStorage"] = laptops_full["ssdStorage"] + laptops_full["hddStorage"]

laptops_full.head()

2024-04-16 23:32:42,433 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-04-16 23:32:42,436 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-16 23:32:42,440 INFO sqlalchemy.engine.Engine select current_schema()
2024-04-16 23:32:42,441 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-16 23:32:42,443 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-04-16 23:32:42,445 INFO sqlalchemy.engine.Engine [raw sql] {}


2024-04-16 23:32:42,554 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 23:32:42,556 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2024-04-16 23:32:42,557 INFO sqlalchemy.engine.Engine [generated in 0.00290s] {'table_name': <sqlalchemy.sql.selectable.Select object at 0x72f699f55900>, 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2024-04-16 23:32:42,594 INFO sqlalchemy.engine.Engine SELECT "Laptop".id, "Laptop".name, "Laptop"."hddStorage", "Laptop"."ssdStorage", "Laptop".ram, "Cpu"."maxSpeed", "Cpu".cores, "G

,id,name,hddStorage,ssdStorage,ram,maxSpeed,cores,gpuMaxSpeed,gpuMemory,directX,openGl,buildNumber,totalStorage
0,1530159,Lenovo Legion 5 Pro 16,0.0,512.0,8.0,4.7,8,1410.0,8.0,12.2,4.6,22000,512.0
1,1259728,ASUS Vivobook 16,0.0,1000.0,8.0,5.0,16,1350.0,0.0,12.1,4.6,22000,1000.0
2,1518251,Lenovo LOQ 15,0.0,512.0,8.0,5.1,8,2130.0,6.0,12.2,4.6,10240,512.0
3,1518264,Lenovo LOQ 15,0.0,512.0,8.0,5.1,8,2130.0,6.0,12.2,4.6,22000,512.0
4,1491011,Lenovo LOQ 15,0.0,512.0,8.0,5.1,8,2130.0,6.0,12.2,4.6,10240,512.0


### GET ALL APPS

In [3]:
app_query = db.select(App.id, App.name, App.minCpuSpeed, App.minCores, App.minDirectX, App.minOpenGl,
                      App.minGpuBoostClock, App.minGpuMemory, App.minRam, App.minStorage, Wind.buildNumber).join(App.windows)
with app.app_context():
    app_full = pd.read_sql(app_query, con=db.engine)

app_full.tail()

2024-04-16 23:32:42,656 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 23:32:42,660 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2024-04-16 23:32:42,663 INFO sqlalchemy.engine.Engine [cached since 0.109s ago] {'table_name': <sqlalchemy.sql.selectable.Select object at 0x72f699feeb30>, 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2024-04-16 23:32:42,673 INFO sqlalchemy.engine.Engine SELECT "Application".id, "Application".name, "Application"."minCpuSpeed", "Application"."minCores", "Application"."minDirectX"

,id,name,minCpuSpeed,minCores,minDirectX,minOpenGl,minGpuBoostClock,minGpuMemory,minRam,minStorage,buildNumber
2386,52f51cb3-a5d1-4c8c-80e8-180c2e96cf08,Forspoken,3.40,6.0,12.05,4.50,589.0,2.048,8.0,50.0,7601
2387,9305e289-41ef-4bb4-a387-cdcdf794632d,Street Fighter 6,3.55,4.0,11.00,3.65,0.0,1.280,8.0,30.0,7601
2388,a34e29b1-2414-4d69-b491-aafa18b8fc2e,Starfield,3.40,6.0,12.05,4.50,589.0,2.048,8.0,50.0,7601
2389,4d421c6b-96c2-49f4-9cd6-15fea9cf669a,Star Wars Jedi: Survivor,3.30,4.0,11.10,4.25,0.0,1.024,8.0,55.0,7601
2390,91bf977d-ef0c-40ac-857e-52805b0b6dd1,Meet Your Maker,3.20,4.0,11.65,4.40,1114.0,2.048,8.0,20.0,10240


### CHOSING REQUIRED APPS

In [4]:
ids = ["976c0e65-db5f-4f0a-ac69-5bccce189c0c"]

app_full[app_full["name"].str.lower().str.contains("devil may")].head()


,id,name,minCpuSpeed,minCores,minDirectX,minOpenGl,minGpuBoostClock,minGpuMemory,minRam,minStorage,buildNumber
68,ff6a43eb-75e9-47e5-b340-2a1b81afc595,Devil May Cry 5,3.20,4.0,11.00,4.60,1032.0,2.000,8.0,35.0,10240
869,51894bd7-ce11-442e-922c-7a24439da756,Devil May Cry 4: Special Edition,2.65,2.0,10.05,2.70,0.0,0.384,2.0,27.0,7601
1106,4fbe4a42-76f2-4e15-994f-f660a9a264ec,DMC: Devil May Cry,2.60,2.0,10.05,2.70,0.0,0.288,2.0,9.0,2600
1264,697d6a71-7cb8-48d0-a5dd-872140bfe08f,Devil May Cry 4,2.40,1.0,9.00,1.75,0.0,0.256,0.0,8.0,2600
2278,6a48c3ed-3a65-42d4-9136-54706d7a3989,Devil May Cry HD Collection,2.65,2.5,11.10,4.30,982.5,2.560,4.0,12.0,7601


In [5]:
app_full.set_index("id", inplace=True)
choosen_apps = app_full.loc[ids]
choosen_apps.head()

KeyError: "['eb7ef6cd-079b-4c20-913e-0844a5d1cac8', '62944821-184e-45d3-b230-61c0cd71c8e1'] not in index"

### find the highest values of each property

In [ ]:
sys_req = pd.DataFrame({
    "maxSpeed": [choosen_apps["minCpuSpeed"].max()],
    "cores": [choosen_apps["minCores"].max()],
    "gpuMaxSpeed": [choosen_apps["minGpuBoostClock"].max()],
    "gpuMemory": [choosen_apps["minGpuMemory"].max()],
    "openGl": [choosen_apps["minOpenGl"].max()],
    "directX": [choosen_apps["minDirectX"].max()],
    "ram": [choosen_apps["minRam"].max()],
    "totalStorage": [choosen_apps["minStorage"].sum()],
    "buildNumber": [choosen_apps["buildNumber"].max()],
})
sys_req.head()

,maxSpeed,cores,gpuMaxSpeed,gpuMemory,openGl,directX,ram,totalStorage,buildNumber
0,3.2,4.0,1032.0,2.0,4.6,11.0,8.0,46.0,10240


### DROP UNUSED LAPTOP PROPERTY

In [ ]:
imp_laptop = laptops_full.drop(columns=["id", "name", "ssdStorage", "hddStorage"])
imp_laptop.head()

,ram,maxSpeed,cores,gpuMaxSpeed,gpuMemory,directX,openGl,buildNumber,totalStorage
0,8.0,4.7,8,1410.0,8.0,12.2,4.6,22000,512.0
1,8.0,5.0,16,1350.0,0.0,12.1,4.6,22000,1000.0
2,8.0,5.1,8,2130.0,6.0,12.2,4.6,10240,512.0
3,8.0,5.1,8,2130.0,6.0,12.2,4.6,22000,512.0
4,8.0,5.1,8,2130.0,6.0,12.2,4.6,10240,512.0


### MERGE AND NORMALIZE SYSTEM REQUIREMENTS DATA AND LAPTOP DATA

In [ ]:
mixed_data = pd.concat([sys_req, imp_laptop], ignore_index=True)
mixed_data.head()

,maxSpeed,cores,gpuMaxSpeed,gpuMemory,openGl,directX,ram,totalStorage,buildNumber
0,3.2,4.0,1032.0,2.0,4.6,11.0,8.0,46.0,10240
1,4.7,8.0,1410.0,8.0,4.6,12.2,8.0,512.0,22000
2,5.0,16.0,1350.0,0.0,4.6,12.1,8.0,1000.0,22000
3,5.1,8.0,2130.0,6.0,4.6,12.2,8.0,512.0,10240
4,5.1,8.0,2130.0,6.0,4.6,12.2,8.0,512.0,22000


In [ ]:
normalized = mixed_data.apply(zscore)
normalized.head()

,maxSpeed,cores,gpuMaxSpeed,gpuMemory,openGl,directX,ram,totalStorage,buildNumber
0,-1.742059,-1.093586,-1.369854,-0.877190,0.360765,-21.487428,-0.822158,-0.793027,-1.499654
1,0.155065,-0.501723,-0.679698,0.420775,0.360765,0.554809,-0.822158,-0.575454,0.666820
2,0.534490,0.682003,-0.789247,-1.309845,0.360765,-1.282044,-0.822158,-0.347609,0.666820
3,0.660965,-0.501723,0.634885,-0.011880,0.360765,0.554809,-0.822158,-0.575454,-1.499654
4,0.660965,-0.501723,0.634885,-0.011880,0.360765,0.554809,-0.822158,-0.575454,0.666820


### CALCULATE EUCLIDEAN DISTANCE

calculate system requirements and laptop distance

In [ ]:
distance = euclidean_distances(normalized, normalized.values[0].reshape(1, -1))

In [ ]:
laptops_full["distance"] = distance[1:]
asc_data = laptops_full.sort_values(by="distance", ascending=True).drop(columns=["ssdStorage", "hddStorage"])

print(sys_req)
asc_data.head()

   maxSpeed  cores  gpuMaxSpeed  gpuMemory  openGl  directX  ram  \
0       3.2    4.0       1032.0        2.0     4.6     11.0  8.0   

   totalStorage  buildNumber  
0          46.0        10240  


,id,name,ram,maxSpeed,cores,gpuMaxSpeed,gpuMemory,directX,openGl,buildNumber,totalStorage,distance
1406,501084,HP Stream 14 (14-ds0000),4.0,2.2,2,686.0,0.0,12.0,4.6,10240,64.0,18.431658
408,927258,HP 14,8.0,3.5,2,1200.0,0.0,12.1,4.6,10240,128.0,20.218107
414,939099,Lenovo IdeaPad 3 15,8.0,3.5,2,1200.0,0.0,12.1,4.6,10240,128.0,20.218107
386,1032676,HP 14,8.0,3.5,2,1200.0,0.0,12.1,4.6,10240,128.0,20.218107
432,723870,HP 15,8.0,3.5,2,1200.0,0.0,12.1,4.6,10240,128.0,20.218107


### FILTER LAPTOP IF LOWER THAN SYSTEM REQUIREMENTS

In [ ]:
baseSpeed = (laptops_full["maxSpeed"] >= sys_req["maxSpeed"].values[0])
cores = (laptops_full["cores"] >= sys_req["cores"].values[0])
gpuMaxSpeed = (laptops_full["gpuMaxSpeed"] >= sys_req["gpuMaxSpeed"].values[0])
directX = (laptops_full["directX"] >= sys_req["directX"].values[0])
openGl = (laptops_full["openGl"] >= sys_req["openGl"].values[0])
totalStorage = (laptops_full["totalStorage"] >= sys_req["totalStorage"].values[0])
buildNumber = (laptops_full["buildNumber"] >= sys_req["buildNumber"].values[0])
vramFromRam = sys_req["gpuMemory"].values[0] - laptops_full["gpuMemory"]
laptops_full["vramFromRam"] = vramFromRam

# if vramFromRam is negative, then vram from ram is not needed (0) alocated vram from ram

laptops_full.loc[laptops_full["vramFromRam"] <= 0, "vramFromRam"] = 0
ram = (laptops_full["ram"] >= (sys_req["ram"].values[0] + laptops_full["vramFromRam"]))

filtered =  laptops_full.loc[baseSpeed & cores & gpuMaxSpeed & directX & openGl & ram & totalStorage & buildNumber]
filtered = filtered.drop(columns=["hddStorage", "ssdStorage"])

print(sys_req)
print(choosen_apps["name"])
filtered.sort_values(by="distance", ascending=True).head()

   maxSpeed  cores  gpuMaxSpeed  gpuMemory  openGl  directX  ram  \
0       3.2    4.0       1032.0        2.0     4.6     11.0  8.0   

   totalStorage  buildNumber  
0          46.0        10240  
id
eb7ef6cd-079b-4c20-913e-0844a5d1cac8    Microsoft Office 2016
976c0e65-db5f-4f0a-ac69-5bccce189c0c                   DOTA 2
62944821-184e-45d3-b230-61c0cd71c8e1          Devil May Cry 5
Name: name, dtype: object


,id,name,ram,maxSpeed,cores,gpuMaxSpeed,gpuMemory,directX,openGl,buildNumber,totalStorage,distance,vramFromRam
657,867442,HP 14,16.0,3.8,4,1500.0,0.0,12.1,4.6,10240,128.0,20.247232,2.0
674,944432,Lenovo ThinkBook 14 Gen 3,16.0,3.8,4,1500.0,0.0,12.1,4.6,10240,256.0,20.247433,2.0
687,968328,HP 14,16.0,3.8,4,1500.0,0.0,12.1,4.6,10240,256.0,20.247433,2.0
686,1104564,HP 17,16.0,3.8,4,1500.0,0.0,12.1,4.6,10240,512.0,20.248365,2.0
691,917774,HP 14,16.0,3.8,4,1500.0,0.0,12.1,4.6,10240,512.0,20.248365,2.0
